# Finding Tuples with Separation 30 kpc < dr < 200 kpc

In [1]:
import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from bigfile import BigFile
import glob, os, struct
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from scipy.interpolate import interp1d

import seaborn as sns

sns.set ()
sns.set_palette ("Set2")
sns.set_style ('ticks', {'ytick.direction' : 'in', 'xtick.direction' : 'in'})

cmap = plt.get_cmap ("Set2")
sns.set_context ("paper", font_scale = 1.7, rc = {"axes.linewidth" : 1.3, "lines.linewidth" : 2.5, "patch.linewidth" : 2.2})
from matplotlib import rcParams as rc
import pickle
import warnings

import import_ipynb
import Tuples_Information as tuples_z2_z3

importing Jupyter notebook from Tuples Information.ipynb


In [2]:
# Specify the cosmological parameters
hh = 0.6774
from colossus.cosmology import cosmology
params = {'flat' : True, 'H0' : 67.74, 'Om0' : 0.2865, 'Ob0' : 0.04628, 'sigma8' : 0.82, 'ns' : 0.96}
cosmo = cosmology.setCosmology ('myCosmo', params)

# some constants and unit conversions
msun_mks = 1.989e30
pc_mks = 3.086e16
grav_mks = 6.67e-11
km_mks = 1e3
yr_mks = 3.154e+7
c_mks = 3e8

Mpc_to_m = 3.086e+22
m_to_mpc = 1.0 / Mpc_to_m
s_to_year = 3.17098e-8
c_Mpc_yr = c_mks * m_to_mpc / s_to_year


# conversion between time and redshift
z_arr = np.linspace (0, 10, 500)
time = cosmo.age (z_arr) # Gyr
def z_to_t (x):
    return interp1d (z_arr, time, fill_value = 'extrapolate')(x)
def t_to_z (x):
    return interp1d(time, z_arr, fill_value = 'extrapolate')(x)

In [3]:
c_mks = 3e8
msun_mks = 2e30
s_to_year = 3.17098e-8
year_to_s = 1./s_to_year
lsun_ergs = 3.9e33
mdot_msun_yr = 1e10/980/1e6
def calc_lx(mdot):
    """
    input: mdot in Msun/yr
    output: Lx in ergs
    """
    lbol = 0.1*mdot*msun_mks/year_to_s*c_mks**2
    lbol_lsun = lbol/3.9e26
    k = 10.83*(lbol_lsun/1e10)**0.28 \
        + 6.08*(lbol_lsun/1e10)**(-0.02)
    return lbol/k*1e7

In [4]:
# BigFile opens up a snapshot to be read
# snapshot 214 stores the z=3 information

# pig = BigFile('/hildafs/datasets/Asterix/PIG2/PIG_348')
pig = BigFile('/hildafs/datasets/Asterix/PIG_files/PIG_214')

BH_IDs = pig.open('5/ID')[:]
BH_pos = pig.open('5/Position')[:]
BH_mass = pig.open('5/BlackholeMass')

FOFGroups_MassByType = pig.open('FOFGroups/MassByType')[:]

# you can check the redshift by reading the attributes of the snapshot
battr = pig["Header"].attrs
scale_fac = battr["Time"][0]
redshift = 1.0 / battr["Time"][0] - 1
print('z =', redshift)

z = 3.0


In [5]:
# pairs_all are selected based on the z coordinate separation
# we did not make any mass or luminosity cut here

# pairs_all = np.load ('/hildafs/projects/phy200018p/share/calvin_2022/pair-m1e7-dr=200kpc-pig348.npy')
pairs_all = np.load ('/hildafs/projects/phy200018p/share/calvin_2022/pair-m1e7-dr=200kpc-pig214.npy')


# now we select pairs with the true separation 30kpc < dr < 200kpc
maskdr = pairs_all['dr'] < 200
maskdr &= pairs_all['dr'] > 30
pairs = pairs_all[maskdr]
print('total pairs with separation 30kpc < dr < 200kpc:', len (pairs))
# lb1 = np.maximum(pairs['lb1'],pairs['lb2'])
# lb2 = np.minimum(pairs['lb1'],pairs['lb2'])

# mass only
mask7 = pairs['m1'] * 1e10 / hh > 1e7
mask7 &= pairs['m2']* 1e10 / hh > 1e7
duals_massive = pairs[mask7]
print ('after mass cut:', len (duals_massive))

total pairs with separation 30kpc < dr < 200kpc: 794
after mass cut: 794


## Triples

In [6]:
length = len(duals_massive)
potential_triples = []

for i in range(length):
    dual = duals_massive[i]
    id1, id2 = dual['id1'], dual['id2']
    
    for j in range(length):
        if i != j:
            itt = duals_massive[j]
            id3, id4 = itt['id1'], itt['id2']
            
            temp = []
            
            if id1 == id3: temp = sorted([id1, id2, id4])
            elif id1 == id4: temp = sorted([id1, id2, id3])
            elif id2 == id3: temp = sorted([id1, id2, id4])
            elif id2 == id4: temp = sorted([id1, id2, id3])
            
            if temp != []: potential_triples.append(temp)

In [7]:
def comp_duals(x, y):
    id1, id2 = x[0], x[1]
    id3, id4 = y[0], y[1]
    
    if sorted([id1, id2]) == sorted([id3, id4]): return True
    
    return False

In [8]:
length = len(potential_triples)
triples = []
dual_ids = duals_massive[['id1','id2']][:]

for i in range(length):
    pot = potential_triples[i]
    id1, id2, id3 = pot[0], pot[1], pot[2]
    
    p1, p2, p3 = (id1, id2), (id1, id3), (id2, id3)
    
    b1, b2, b3 = False, False, False
    
    for dual in dual_ids:
        if comp_duals(p1, dual): b1 = True
        elif comp_duals(p2, dual): b2 = True
        elif comp_duals(p3, dual): b3 = True
            
    if False not in [b1, b2, b3]:
        if sorted(pot) not in triples:
            triples.append(sorted(pot))

In [9]:
print('total potential triples with separation 30kpc < dr < 200kpc:', len(potential_triples))
print('total triples with separation 30kpc < dr < 200kpc (with potential quadruples):', len(triples))

total potential triples with separation 30kpc < dr < 200kpc: 1176
total triples with separation 30kpc < dr < 200kpc (with potential quadruples): 126


## Quadruples

In [10]:
length = len(triples)
potential_quadruples = []

for i in range(length):
    trip = triples[i]
    id1, id2, id3 = trip[0], trip[1], trip[2]
    
    for j in range(length):
        if i != j:
            itt = triples[j]
            id4, id5, id6 = itt[0], itt[1], itt[2]
            
            temp = []
            
            if (id1 == id4):
                if (id2 == id5):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id2 == id6):
                    temp = sorted ((id1, id2, id3, id5))
                elif (id3 == id5):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id3 == id6):
                    temp = sorted ((id1, id2, id3, id5))
            elif (id2 == id4):
                if (id1 == id5):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id1 == id6):
                    temp = sorted ((id1, id2, id3, id5))
                elif (id3 == id5):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id3 == id6):
                    temp = sorted ((id1, id2, id3, id5))
            elif (id3 == id4):
                if (id1 == id5):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id1 == id6):
                    temp = sorted ((id1, id2, id3, id5))
                elif (id2 == id5):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id2 == id6):
                    temp = sorted ((id1, id2, id3, id5))
                        
                
            elif (id1 == id5):
                if (id2 == id4):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id2 == id6):
                    temp = sorted ((id1, id2, id3, id4))
                elif (id3 == id4):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id3 == id6):
                    temp = sorted ((id1, id2, id3, id4))
            elif (id2 == id5):
                if (id1 == id4):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id1 == id6):
                    temp = sorted ((id1, id2, id3, id4))
                elif (id3 == id4):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id3 == id6):
                    temp = sorted ((id1, id2, id3, id4))
            elif (id3 == id5):
                if (id1 == id4):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id1 == id6):
                    temp = sorted ((id1, id2, id3, id4))
                elif (id2 == id4):
                    temp = sorted ((id1, id2, id3, id6))
                elif (id2 == id6):
                    temp = sorted ((id1, id2, id3, id4))
                        
            elif (id1 == id6):
                if (id2 == id4):
                    temp = sorted (id1, id2, id3, id5)
                elif (id2 == id5):
                    temp = sorted (id1, id2, id3, id4)
                elif (id3 == id4):
                    temp = sorted (id1, id2, id3, id5)
                elif (id3 == id5):
                    temp = sorted (id1, id2, id3, id4)
            elif (id2 == id6):
                if (id1 == id4):
                    temp = sorted (id1, id2, id3, id5)
                elif (id1 == id5):
                    temp = sorted (id1, id2, id3, id4)
                elif (id3 == id4):
                    temp = sorted (id1, id2, id3, id5)
                elif (id3 == id5):
                    temp = sorted (id1, id2, id3, id4)
            elif (id3 == id6):
                if (id1 == id4):
                    temp = sorted (id1, id2, id3, id5)
                elif (id1 == id5):
                    temp = sorted (id1, id2, id3, id4)
                elif (id2 == id4):
                    temp = sorted (id1, id2, id3, id5)
                elif (id2 == id5):
                    temp = sorted (id1, id2, id3, id4)
            
            if temp != []: potential_quadruples.append(temp)

In [11]:
def comp_trips(x, y):
    id1, id2, id3 = x[0], x[1], x[2]
    id4, id5, id6 = y[0], y[1], y[2]
    
    if sorted([id1, id2, id3]) == sorted([id4, id5, id6]): return True
    
    return False

In [12]:
length = len(potential_quadruples)
quadruples = []

for i in range(length):
    pot = potential_quadruples[i]
    id1, id2, id3, id4 = pot[0], pot[1], pot[2], pot[3]
    
    p1, p2, p3, p4 = (id1, id2, id3), (id1, id2, id4), (id1, id3, id4), (id2, id3, id4)
    
    b1, b2, b3, b4 = False, False, False, False
    
    for trip in triples:
        if comp_trips(p1, trip): b1 = True
        elif comp_trips(p2, trip): b2 = True
        elif comp_trips(p3, trip): b3 = True
        elif comp_trips(p4, trip): b4 = True        
            
    if False not in [b1, b2, b3, b4]:
        if sorted(pot) not in quadruples:
            quadruples.append(sorted(pot))

In [13]:
print('total potential quadruples with separation 30kpc < dr < 200kpc:', len(potential_quadruples))
print('total quadruples with separation 30kpc < dr < 200kpc (with potential quadruples):', len(quadruples))

total potential quadruples with separation 30kpc < dr < 200kpc: 406
total quadruples with separation 30kpc < dr < 200kpc (with potential quadruples): 24


## Quintuples

In [14]:
def intersection(lst1, lst2):
    return list(set(lst1) & set(lst2))

In [15]:
length = len(quadruples)
potential_quintuples = []

for i in range(length):
    quad1 = sorted(quadruples[i])
    
    for j in range(length):
        if i != j:
            quad2 = sorted(quadruples[j])
            
            temp = []
            
            sect = intersection(quad1, quad2)
            
            if len(sect) == 3:
                temp = [curr for curr in sect]
                
                for curr in quad1:
                    if curr not in temp: temp.append(curr)
                for curr in quad2:
                    if curr not in temp: temp.append(curr)
                
                assert(len(temp) == 5)
            
            if temp != []: potential_quintuples.append(temp)

In [16]:
def comp_quads(x, y):
    id1, id2, id3, id4 = x[0], x[1], x[2], x[3]
    id5, id6, id7, id8 = y[0], y[1], y[2], y[3]
    
    if sorted([id1, id2, id3, id4]) == sorted([id5, id6, id7, id8]): return True
    
    return False

In [17]:
length = len(potential_quintuples)
quintuples = []

for i in range(length):
    pot = potential_quintuples[i]
    id1, id2, id3, id4, id5 = pot[0], pot[1], pot[2], pot[3], pot[4]
    
    p1, p2, p3, p4, p5 = (id1, id2, id3, id4), (id1, id2, id4, id5), (id1, id3, id4, id5), (id2, id3, id4, id5), (id1, id2, id3, id5)
    
    b1, b2, b3, b4, b5 = False, False, False, False, False
    
    for quad in quadruples:
        if comp_quads(p1, quad): b1 = True
        elif comp_quads(p2, quad): b2 = True
        elif comp_quads(p3, quad): b3 = True
        elif comp_quads(p4, quad): b4 = True   
        elif comp_quads(p5, quad): b5 = True
            
    if False not in [b1, b2, b3, b4, b5]:
        if sorted(pot) not in quintuples:
            quintuples.append(sorted(pot))

### Removing Quadruple Subsets from Triples

In [18]:
unique_triples = []

for trip in triples:
    id1, id2, id3 = trip[0], trip[1], trip[2]
    isQuad = False
    
    for quad in quadruples:
        if id1 in quad and id2 in quad and id3 in quad: isQuad = True
    
    if not isQuad: unique_triples.append(trip)

In [19]:
print('total triples without filtering out quadruple sub-tuples:', len(triples))
print('total triples not within quadruples:', len(unique_triples))

triples = unique_triples

total triples without filtering out quadruple sub-tuples: 126
total triples not within quadruples: 64


### Removing Quintuple Subsets from Quadruples

In [20]:
unique_quadruples = []

for quad in quadruples:
    id1, id2, id3, id4 = quad[0], quad[1], quad[2], quad[3]
    isQuint = False
    
    for quint in quintuples:
        if id1 in quint and id2 in quint and id3 in quint and id4 in quint: isQuint = True
    
    if not isQuint: unique_quadruples.append(quad)

In [21]:
print('total quadruples without filtering out quintuple sub-tuples:', len(quadruples))
print('total quadruples not within quintuples:', len(unique_quadruples))

quadruples = unique_quadruples

total quadruples without filtering out quintuple sub-tuples: 24
total quadruples not within quintuples: 10


## Final IDs of Tuples

In [22]:
triples

[[190330923700, 190754027667, 191661329745],
 [190330923700, 190754027667, 191117390607],
 [190300387606, 190330923700, 191661329745],
 [190330923700, 191661329745, 192266720145],
 [190330923700, 191117390607, 192266720145],
 [202057799372, 202117897875, 202299903826],
 [201603851309, 202057799372, 202299903826],
 [288759510889, 290392807456, 290967678486],
 [288366189469, 288759510889, 290392807456],
 [319384488647, 319868472095, 320472960611],
 [318930419617, 319868472095, 320472960611],
 [319868472095, 320201040644, 320472960611],
 [318779164189, 319868472095, 320472960611],
 [328855687577, 329763440569, 331911201567],
 [328855687577, 329763440569, 331517869025],
 [328855687577, 329763440569, 332697487041],
 [182041328590, 182918567646, 183251757629],
 [181618224650, 182918567646, 183251757629],
 [182041328590, 183251757629, 183584172159],
 [181618224650, 183251757629, 183584172159],
 [294241478728, 295391110693, 297992407150],
 [283143487325, 284444402325, 284958547835],
 [28114699

In [23]:
quadruples

[[188575103637, 188605810153, 188847689217, 190783793705],
 [188575103637, 188605810153, 189664114719, 190783793705],
 [188575103637, 188605810153, 188635972273, 190783793705],
 [269000537783, 269725954826, 269847680861, 270905979821],
 [301356899793, 302173721354, 302748113800, 303655767889],
 [214843751341, 215418721290, 216054251780, 216145089724],
 [281146998310, 281207404777, 281540435352, 282145473814],
 [326353608267, 327321437652, 327442657705, 327714621701],
 [281593215811, 281835463287, 283377954772, 283710699326],
 [281774523287, 281835463287, 283377954772, 283710699326]]

## Generating BH Information

In [24]:
import h5py

In [25]:
# z=3 snap: 214
# z=2 snap: 348

snap = 214

In [26]:
if snap > 294:
    pig = BigFile('/hildafs/datasets/Asterix/PIG2/PIG_%03d'%snap)
else:
    pig = BigFile('/hildafs/datasets/Asterix/PIG_files/PIG_%03d'%snap)

# you can check the redshift by reading the attributes of the snapshot
battr = pig["Header"].attrs
scale_fac = battr["Time"][0]
redshift = 1./battr["Time"][0] - 1
Lbox = battr['BoxSize']
hh = battr['HubbleParam']
om0 = battr['Omega0']
omb = battr['OmegaBaryon']
oml = battr['OmegaLambda']
Nfof = battr['NumFOFGroupsTotal']
sigma8 = 0.82
print('----------PIG file info------------')

print('Redshift = %.2f'%redshift)
print('Lbox = %d ckpc/h'%Lbox)
print('NfofGroups = %d'%Nfof)

print('------cosmological parameters-----')
print('h = %.4f'%hh)
print('Omega_m = %.4f'%om0)
print('Omega_b = %.4f'%omb)
print('Omega_l = %.4f'%oml)

----------PIG file info------------
Redshift = 3.00
Lbox = 250000 ckpc/h
NfofGroups = 193500739
------cosmological parameters-----
h = 0.6774
Omega_m = 0.3089
Omega_b = 0.0486
Omega_l = 0.6911


In [27]:
# This helps you locate particles of each type if you know the group index
Length = pig.open('FOFGroups/LengthByType')[:]
OffsetByType = np.cumsum(Length,axis=0)
a1 = np.array([[0,0,0,0,0,0]],dtype=np.uint64)
OffsetByType = np.append(a1,OffsetByType,axis=0)

In [28]:
# "5" means BH particles, "ID" means we are loading the ID of each BH particle
# here we only load the first 1000000, because most of the massive BHs are there
BHIDs = pig.open('5/ID')[:]

# pick a fairly massive black hole as an example
id1 = 258052176737
idx1 = (BHIDs==id1).nonzero()[0][0]
print('ID of the BH of interest:',id1)

ID of the BH of interest: 258052176737


In [29]:
idxdir = '/hildafs/datasets/Asterix/subfind/subfind-idx/subfind_%03d/'%snap
bf = BigFile(idxdir)

In [30]:
def get_bh_group(pig,bid):
    bhidx = (BHIDs==bid).nonzero()[0][0]
    groupidx = pig.open('5/GroupID')[bhidx]-1
    subidx = sfile.open('5/Subfind-SubGrpIndex')[bhidx]
    return bhidx,groupidx,subidx

In [31]:
def get_subfind_chunk(snap):
    subdir = '/hildafs/datasets/Asterix/subfind/subfind_%d/*'%snap
    chunk_list = []
    maxgroup_list = []
    for ff in sorted(glob.glob(subdir)):
        cname = ff.split('/')[-1]
        chunk = int(cname.split('.')[0][5:])
        maxgroup = int(cname.split('.')[1])
        chunk_list.append(chunk)
        maxgroup_list.append(maxgroup)
    sort = np.argsort(chunk_list)
    
    chunk_list = np.array(chunk_list)[sort]
    maxgroup_list = np.array(maxgroup_list)[sort]
    return chunk_list, maxgroup_list 

In [32]:
chunk_list, maxgroup_list = get_subfind_chunk(snap)
def group_chunk_dir(groupidx,snap):
    sub_root = '/hildafs/datasets/Asterix/subfind/subfind_%03d/'%snap
    chunk = np.nonzero(maxgroup_list-1>=groupidx)[0][0]
    # print('groupidx',groupidx,'chunk',chunk,maxgroup_list[chunk-1],maxgroup_list[chunk])
    
    subdir = sub_root + 'chunk%d.%d/output/'%(chunk,maxgroup_list[chunk])
    tabfile = subdir + 'fof_subhalo_tab_%03d.hdf5'%snap
    return subdir

In [33]:
sfile = BigFile('/hildafs/datasets/Asterix/subfind/subfind-idx/subfind_%03d'%snap)
subdir = group_chunk_dir(groupidx=10,snap=snap)
tabfile = subdir + 'fof_subhalo_tab_%03d.hdf5'%snap
with h5py.File(tabfile,'r') as sbgrp:
    print(sbgrp.keys())
    print('\n ---- Available Group Properties --------')
    print(list(sbgrp['Group'].keys()))
    print('\n ---- Available Subhalo Properties --------')
    print(list(sbgrp['Subhalo'].keys()))

<KeysViewHDF5 ['Config', 'Group', 'Header', 'IDs', 'Parameters', 'Subhalo']>

 ---- Available Group Properties --------
['GroupAscale', 'GroupFirstSub', 'GroupLen', 'GroupLenType', 'GroupMass', 'GroupMassType', 'GroupNsubs', 'GroupOffsetType', 'GroupPos', 'GroupVel', 'Group_M_Crit200', 'Group_M_Crit500', 'Group_M_Mean200', 'Group_M_TopHat200', 'Group_R_Crit200', 'Group_R_Crit500', 'Group_R_Mean200', 'Group_R_TopHat200']

 ---- Available Subhalo Properties --------
['SubhaloCM', 'SubhaloGroupNr', 'SubhaloHalfmassRad', 'SubhaloHalfmassRadType', 'SubhaloIDMostbound', 'SubhaloLen', 'SubhaloLenType', 'SubhaloMass', 'SubhaloMassType', 'SubhaloOffsetType', 'SubhaloParentRank', 'SubhaloPos', 'SubhaloRankInGr', 'SubhaloSpin', 'SubhaloVel', 'SubhaloVelDisp', 'SubhaloVmax', 'SubhaloVmaxRad']


In [34]:
def subfind_obt(groupidx,snap):
    subdir = group_chunk_dir(groupidx,snap)
    tabfile = subdir + 'fof_subhalo_tab_%03d.hdf5'%snap
    
    with h5py.File(tabfile,'r') as sbgrp:
        zeros = np.array([[0,0,0,0,0,0]],dtype=np.uint64)

        glbt = sbgrp['Group']['GroupLenType'][:]
        gobt = np.concatenate([zeros,np.cumsum(glbt,axis=0)],axis=0).astype(int)


        first_sub = sbgrp['Group']['GroupFirstSub'][:]
        nsub = sbgrp['Group']['GroupNsubs'][:]

        # print('total subhalos in this chunk:',sum(nsub))


        slbt = sbgrp['Subhalo']['SubhaloLenType'][:]
        sobt = np.zeros_like(slbt)
        sobt = np.concatenate([zeros,sobt],axis=0).astype(int)
        for i,f in enumerate(first_sub):
            if f < 0:  # skip groups with no subhalo
                continue
            # align the first subgroup with group starting point
            sobt[f] = gobt[i]
            # assign the rest of the subgroup starting idx
            sobt[f+1:f+nsub[i]] = sobt[f] + np.cumsum(slbt[f:f+nsub[i]-1],axis=0)
            
        return gobt,sobt,first_sub

In [35]:
def place_bh(groupidx,bhid,snap,gobt,sobt,first_sub):
    subdir = group_chunk_dir(groupidx,snap)
    if snap < 294:
        grpfile = subdir + 'snap_%03d.hdf5'%snap
    else:
        grpfile = subdir + 'snap-groupordered_%03d.hdf5'%snap
    
    with h5py.File(grpfile,'r') as sbgrp:
        id5 = sbgrp['PartType5']['ParticleIDs'][:]
        bidx = (id5==bhid).nonzero()[0][0]
        
    gidx = (gobt[:,5]>bidx).nonzero()[0][0]-1
    
    sidx = -1
    
    if len((sobt[:,5]>bidx).nonzero()) > 0:
        if len((sobt[:,5]>bidx).nonzero()[0]) > 0:
            sidx = (sobt[:,5]>bidx).nonzero()[0][0]-1

    if sidx == -1:
        print("Missing BHID:",bhid)
        return bidx,gidx,sidx,0,0
        
    # make sure that we get the correct BH group
    bh_group = id5[gobt[gidx][5]:gobt[gidx+1][5]]
    bh_sbgrp = id5[sobt[sidx][5]:sobt[sidx+1][5]]
    assert(bhid in bh_group)
    assert(bhid in bh_sbgrp)
    
    sbegin = first_sub[gidx]
    send = first_sub[gidx+1]

    return bidx,gidx,sidx,sbegin,send

In [36]:
def group_summary(groupidx,snap,gidx,feature_list):
    subdir = group_chunk_dir(groupidx,snap)
    tabfile = subdir + 'fof_subhalo_tab_%03d.hdf5'%snap
    
    output = {}
    with h5py.File(tabfile,'r') as sbgrp:
        for ff in feature_list:
            try: 
                output[ff] = sbgrp['Group'][ff][gidx]
            except KeyError:
                print('skipping %s: feature does not exist!'%ff) 
    return output

In [37]:
def subhalo_summary(groupidx,snap,gidx,sidx,sbegin,send,feature_list,main_sub=False):
    subdir = group_chunk_dir(groupidx,snap)
    tabfile = subdir + 'fof_subhalo_tab_%03d.hdf5'%snap
    
    if main_sub:
        print('output the main subhalo in group')
        sidx = sbegin

    
    output = {}
    with h5py.File(tabfile,'r') as sbgrp:
        for ff in feature_list:
            try: 
                output[ff] = sbgrp['Subhalo'][ff][sidx]
            except KeyError:
                print('skipping %s: feature does not exist!'%ff)
    return output

In [38]:
# These arrays are necessary for locating a BH in the detials file
path = '/hildafs/datasets/Asterix/BH_details_dict/Read-Blackhole-Detail'
detail = BigFile(path)
AllIDs = detail.open('BHID')[:]
Index = detail.open('Index')[:]

In [39]:
def get_bh_history(bhid):
    idx = (AllIDs==bhid).nonzero()[0][0]
    chunk = Index[idx]
    # print('File number of the target BHs:',chunk)
    # now load bh data
    outdir = '/hildafs/datasets/Asterix/BH_details_dict/'
    save = outdir+'BlackholeDetails-%04d'%chunk
    with open(save, 'rb') as f:
        data = pickle.load(f)
        f.close()
    bh = data[bhid]
    return bh

### Triples Information

In [40]:
triples = tuples_z2_z3.triples_z2_dr200
quadruples = tuples_z2_z3.quadruples_z2_dr200

In [41]:
trips_vals = []

count = 1

for trip in triples:
    
    if count % (len(triples)//10) == 0:
        print("Count =", count)
    
    count += 1
    
    for id1 in trip:
        
        # id1 = 258052176737
        bidx_bf,gidx_bf,sidx_bf = get_bh_group(pig,id1)

        start,end = OffsetByType[gidx_bf],OffsetByType[gidx_bf+1]

        gobt,sobt,first_sub = subfind_obt(groupidx=gidx_bf,snap=snap)

        bidx,gidx,sidx,sbegin,send = place_bh(gidx_bf,bhid=id1,snap=snap,gobt=gobt,sobt=sobt,first_sub=first_sub)

        # look out for reassigned subhalo on some large BHs
        sub_info = subhalo_summary(groupidx=gidx_bf,snap=snap,gidx=gidx,sidx=sidx,\
                                   sbegin=sbegin,send=send,feature_list=['SubhaloPos','SubhaloMassType'],main_sub=False)

        gal_center = sub_info['SubhaloPos']
        gal_mass = sub_info['SubhaloMassType'][4]*1e10/hh
        bhmass = pig.open('5/BlackholeMass')[bidx_bf]*1e10/hh[0]

        if gal_mass < bhmass:
            # reassign galaxy
            mass4 = pig.open('4/Mass')[start[4]:end[4]]
            star_sid = bf.open('4/Subfind-SubGrpIndex2')[start[4]:end[4]]
            bh_sbgrp = bf.open('5/Subfind-SubGrpIndex2')[bidx_bf]
            mask4_tgt = star_sid==bh_sbgrp

            gal_mass = np.sum(mass4[mask4_tgt])*1e10/hh

        bhpos = pig.open('5/Position')[bidx_bf]
        
        x1,y1,z1 = bhpos[0],bhpos[1],bhpos[2]
        x2,y2,z2 = gal_center[0],gal_center[1],gal_center[2]
        
        offset = (math.sqrt ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2) / (1 + redshift) / hh)[0]
        
        bh1 = get_bh_history(id1)
        
        mask1 = bh1['z'] == redshift
        
        bh1 = bh1[mask1]
        
        lum1 = calc_lx(bh1['Mdot']*mdot_msun_yr)[0]
        
        curr = (id1, bhmass, gal_mass[0], offset, lum1)
        trips_vals.append(curr)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
trips_info_z2_dr200 = np.array(trips_vals,
    dtype = [('BHID', '<i8'), ('BHMass', '<f8'), ('HostMass', '<f8'), ('Offset', '<f8'), ('LumX', '<f8')])

In [ ]:
len(trips_info_z2_dr200)

In [ ]:
trips_info_z2_dr200

### Quadruples Information

In [ ]:
quads_vals = []
missing_quads = []
count = 1

for quad in quadruples:
    
    if count % (len(quadruples)//10) == 0:
        print("Count =", count)
        
    count += 1
        
    for id1 in quad:
        # id1 = 258052176737
        
        bidx_bf,gidx_bf,sidx_bf = get_bh_group(pig,id1)

        start,end = OffsetByType[gidx_bf],OffsetByType[gidx_bf+1]

        gobt,sobt,first_sub = subfind_obt(groupidx=gidx_bf,snap=snap)

        bidx,gidx,sidx,sbegin,send = place_bh(gidx_bf,bhid=id1,snap=snap,gobt=gobt,sobt=sobt,first_sub=first_sub)
        
        if sidx == -1:
            missing_quads.append(id1)
        else:
            # look out for reassigned subhalo on some large BHs
            sub_info = subhalo_summary(groupidx=gidx_bf,snap=snap,gidx=gidx,sidx=sidx,\
                                       sbegin=sbegin,send=send,feature_list=['SubhaloPos','SubhaloMassType'],main_sub=False)

            gal_center = sub_info['SubhaloPos']
            gal_mass = sub_info['SubhaloMassType'][4]*1e10/hh
            bhmass = pig.open('5/BlackholeMass')[bidx_bf]*1e10/hh[0]

            if gal_mass < bhmass:
                # reassign galaxy
                mass4 = pig.open('4/Mass')[start[4]:end[4]]
                star_sid = bf.open('4/Subfind-SubGrpIndex2')[start[4]:end[4]]
                bh_sbgrp = bf.open('5/Subfind-SubGrpIndex2')[bidx_bf]
                mask4_tgt = star_sid==bh_sbgrp

                gal_mass = np.sum(mass4[mask4_tgt])*1e10/hh

            bhpos = pig.open('5/Position')[bidx_bf]

            x1,y1,z1 = bhpos[0],bhpos[1],bhpos[2]
            x2,y2,z2 = gal_center[0],gal_center[1],gal_center[2]

            offset = (math.sqrt ((x2 - x1)**2 + (y2 - y1)**2 + (z2 - z1)**2) / (1 + redshift) / hh)[0]

            bh1 = get_bh_history(id1)

            mask1 = bh1['z'] == redshift

            bh1 = bh1[mask1]

            lum1 = calc_lx(bh1['Mdot']*mdot_msun_yr)[0]

            curr = (id1, bhmass, gal_mass[0], offset, lum1)
            quads_vals.append(curr)

In [ ]:
quads_info_z2_dr200 = np.array(quads_vals,
    dtype = [('BHID', '<i8'), ('BHMass', '<f8'), ('HostMass', '<f8'), ('Offset', '<f8'), ('LumX', '<f8')])

In [ ]:
len(quads_info_z2_dr200)/4

In [ ]:
quads_info_z2_dr200

In [ ]:
missing_quads